In [1]:
from google.colab import drive
import numpy as np
import seaborn as sns
import zipfile
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
zip_file_path = '/content/drive/MyDrive/Teeth_Dataset2.zip'
extract_dir = '/content/Teeth DataSet/'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Files extracted to:", extract_dir)

Files extracted to: /content/Teeth DataSet/


In [3]:
train_dir = '/content/Teeth DataSet/Teeth_Dataset/Training'
validation_dir = '/content/Teeth DataSet/Teeth_Dataset/Validation'
test_dir = '/content/Teeth DataSet/Teeth_Dataset/Testing'
# Image dimensions and batch size
IMG_HEIGHT = 224  # EfficientNetB0 expects 224x224 images
IMG_WIDTH = 224
BATCH_SIZE = 32

#2. Data Augmentation and Normalization

In [4]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

validation_image_generator = ImageDataGenerator(rescale=1./255)

# Load data
train_data_gen = train_image_generator.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_data_gen = validation_image_generator.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data_gen = validation_image_generator.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 3087 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


#3. Model Architecture and Training

In [5]:
# Load ResNet50 model with pre-trained
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
x = Dense(train_data_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy',  metrics=['accuracy'])
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [7]:
# Train the model
history = model.fit( train_data_gen, epochs=30, steps_per_epoch=train_data_gen.samples // BATCH_SIZE,
    validation_data=val_data_gen, validation_steps=val_data_gen.samples // BATCH_SIZE, callbacks=[lr_scheduler, early_stopping])

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


96/96 ━━━━━━━━━━━━━━━━━━━━ 75s 584ms/step - accuracy: 0.1639 - loss: 5.1973 - val_accuracy: 0.1758 - val_loss: 4.0147 - learning_rate: 1.0000e-04
Epoch 2/30
 1/96 ━━━━━━━━━━━━━━━━━━━━ 6s 73ms/step - accuracy: 0.1562 - loss: 4.0981

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


96/96 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.1562 - loss: 4.0981 - val_accuracy: 0.0000e+00 - val_loss: 4.2446 - learning_rate: 1.0000e-04
Epoch 3/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 50s 479ms/step - accuracy: 0.1766 - loss: 3.8874 - val_accuracy: 0.2100 - val_loss: 3.3129 - learning_rate: 1.0000e-04
Epoch 4/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.0938 - loss: 3.4288 - val_accuracy: 0.0000e+00 - val_loss: 3.4370 - learning_rate: 1.0000e-04
Epoch 5/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 78s 479ms/step - accuracy: 0.1675 - loss: 3.2637 - val_accuracy: 0.2295 - val_loss: 2.9669 - learning_rate: 1.0000e-04
Epoch 6/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1875 - loss: 2.8965 - val_accuracy: 0.5000 - val_loss: 2.8473 - learning_rate: 1.0000e-04
Epoch 7/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 81s 480ms/step - accuracy: 0.1614 - loss: 2.9554 - val_accuracy: 0.2275 - val_loss: 2.8029 - learning_rate: 1.0000e-04
Epoch 8/30
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.156

In [8]:
# Unfreeze more layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True
# lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
history_finetune = model.fit(train_data_gen, epochs=20, steps_per_epoch=train_data_gen.samples // BATCH_SIZE, validation_data=val_data_gen, validation_steps=val_data_gen.samples // BATCH_SIZE,
    callbacks=[lr_scheduler, early_stopping])

test_loss, test_acc = model.evaluate(test_data_gen)
print(f'Test Accuracy: {test_acc}')

Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 83s 611ms/step - accuracy: 0.1778 - loss: 2.6089 - val_accuracy: 0.1787 - val_loss: 2.6727 - learning_rate: 1.0000e-05
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - accuracy: 0.3125 - loss: 2.4734 - val_accuracy: 0.5000 - val_loss: 2.5546 - learning_rate: 1.0000e-05
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 53s 480ms/step - accuracy: 0.2308 - loss: 2.5076 - val_accuracy: 0.1904 - val_loss: 2.5953 - learning_rate: 1.0000e-05
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2500 - loss: 2.4006 - val_accuracy: 0.2500 - val_loss: 2.3414 - learning_rate: 1.0000e-05
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 49s 480ms/step - accuracy: 0.2586 - loss: 2.4418 - val_accuracy: 0.2451 - val_loss: 2.4414 - learning_rate: 1.0000e-05
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.4062 - loss: 2.3516 - val_accuracy: 0.2500 - val_loss: 2.3798 - learning_rate: 1.0000e-05
Epoch 7/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 81s 481ms/step - accuracy: